<a href="https://colab.research.google.com/github/yukismd/DL_for_ImageData_and_Finetuning/blob/main/text_classification_hatespeech/PT/luke-japanese-large-lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
MODEL = 'studio-ousia/luke-japanese-large-lite'

In [2]:
!pwd

/content


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls

drive  sample_data


In [5]:
!ls drive/MyDrive/experiment_results

hs_txt_classification


In [6]:
SAVE_DIR = 'drive/MyDrive/experiment_results/hs_txt_classification'

In [7]:
!pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [8]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
#!pip install fugashi unidic_lite

In [9]:
import datasets
print(datasets.__version__)
import transformers
print(transformers.__version__)
from transformers import AutoTokenizer
import evaluate
print(evaluate.__version__)

import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import json

2.19.1
4.41.1
0.4.2


## 学習データ, テストデータ

In [10]:
df = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_train.csv')
print(df.shape)

(4656, 5)


In [11]:
df_train = df[df['hold'] != 0]    # 学習データ
df_valid = df[df['hold'] == 0]    # 検証データ
df_train.shape, df_valid.shape

((3759, 5), (897, 5))

In [12]:
ds_train = datasets.Dataset.from_pandas(df_train[['text','label']], preserve_index=False)
ds_valid = datasets.Dataset.from_pandas(df_valid[['text','label']], preserve_index=False)

In [13]:
df_test = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_test.csv')
print(df_test.shape)

(600, 4)


In [14]:
ds_test = datasets.Dataset.from_pandas(df_test[['text','label']], preserve_index=False)

## トークナイザー

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/842k [00:00<?, ?B/s]

entity_vocab.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

MLukeTokenizer(name_or_path='studio-ousia/luke-japanese-large-lite', vocab_size=32770, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<ent>', '<ent2>', '<ent>', '<ent2>', '<ent>', '<ent2>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32769: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=T

In [16]:
def tokenize_function(examples):
    return tokenizer(examples["text"],
                     truncation=True,
                     max_length=128)    # 学習データの文長が短いので128に設定

In [17]:
ds_tkn_train = ds_train.map(tokenize_function, batched=True)
ds_tkn_valid = ds_valid.map(tokenize_function, batched=True)
ds_tkn_train, ds_tkn_valid

Map:   0%|          | 0/3759 [00:00<?, ? examples/s]

Map:   0%|          | 0/897 [00:00<?, ? examples/s]

(Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 3759
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 897
 }))

In [18]:
# textカラムの削除
ds_tkn_train = ds_tkn_train.remove_columns(['text'])
ds_tkn_valid = ds_tkn_valid.remove_columns(['text'])

# labelカラムをlabelsへリネーム
ds_tkn_train = ds_tkn_train.rename_column('label', 'labels')
ds_tkn_valid = ds_tkn_valid.rename_column('label', 'labels')

In [19]:
ds_tkn_test = ds_test.map(tokenize_function, batched=True)
ds_tkn_test

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 600
})

In [20]:
# textカラムの削除
ds_tkn_test = ds_tkn_test.remove_columns(['text'])

# labelカラムをlabelsへリネーム
ds_tkn_test = ds_tkn_test.rename_column('label', 'labels')

## 学習

In [21]:
metric = evaluate.load("f1")

# 精度評価関数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
def do_training():
  print('N_EPOCH: ', N_EPOCH)
  print('BATCH_SIZE: ', BATCH_SIZE)
  print('LR', LR)

  model = AutoModelForSequenceClassification.from_pretrained(MODEL,
                                                           num_labels=2)

  trainer_args = TrainingArguments(
    seed=np.random.randint(0, 9999),
    output_dir=SAVE_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    #save_strategy="epoch",
    save_steps=1e6, # saveのステップを大きくしてここではモデルの保存を行わないようにする
    log_level="critical",
    num_train_epochs=N_EPOCH,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=12,
    save_total_limit=1,
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
    )

  trainer = Trainer(
    model=model,
    args=trainer_args,
    tokenizer=tokenizer,
    train_dataset=ds_tkn_train,
    eval_dataset=ds_tkn_valid,
    compute_metrics=compute_metrics,
    )

  #return trainer

  print('----------  Start Training  ----------')
  trainer.train()

  print('----------  Res on Valid  ----------')
  res_json = trainer.evaluate()
  print(res_json)

  print('----------  Res on Test  ----------')
  test_preds = trainer.predict(ds_tkn_test)
  test_metric = test_preds.metrics
  print(test_metric)

  return res_json, test_metric


In [24]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.000003  # 3e-6

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch16_Lr3e-06
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-06


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1889, 'grad_norm': 0.17683807015419006, 'learning_rate': 2.0170212765957445e-06, 'epoch': 1.0}
{'eval_loss': 0.16933223605155945, 'eval_f1': 0.5070422535211268, 'eval_runtime': 3.0481, 'eval_samples_per_second': 294.286, 'eval_steps_per_second': 24.606, 'epoch': 1.0}
{'loss': 0.112, 'grad_norm': 28.99391746520996, 'learning_rate': 1.0170212765957446e-06, 'epoch': 2.0}
{'eval_loss': 0.1251472532749176, 'eval_f1': 0.6153846153846154, 'eval_runtime': 2.677, 'eval_samples_per_second': 335.079, 'eval_steps_per_second': 28.017, 'epoch': 2.0}
{'loss': 0.0814, 'grad_norm': 1.056697130203247, 'learning_rate': 2.127659574468085e-08, 'epoch': 3.0}
{'eval_loss': 0.14322562515735626, 'eval_f1': 0.6481481481481481, 'eval_runtime': 2.6997, 'eval_samples_per_second': 332.256, 'eval_steps_per_second': 27.781, 'epoch': 3.0}
{'train_runtime': 219.8478, 'train_samples_per_second': 51.295, 'train_steps_per_second': 3.207, 'train_loss': 0.12746067452937998, 

In [25]:
# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

drive/MyDrive/experiment_results/hs_txt_classification/VALID__luke-japanese-large-lite_Epoch3_Batch16_Lr3e-06.json
drive/MyDrive/experiment_results/hs_txt_classification/TEST__luke-japanese-large-lite_Epoch3_Batch16_Lr3e-06.json


In [26]:
!ls drive/MyDrive/experiment_results/hs_txt_classification

bert-large-japanese-v2	OLD2
checkpoint-940		TEST__luke-japanese-large-lite_Epoch3_Batch16_Lr3e-06.json
japanese-roberta-base	VALID__luke-japanese-large-lite_Epoch3_Batch16_Lr3e-06.json
OLD


In [27]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch16_Lr1e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1551, 'grad_norm': 11.07096004486084, 'learning_rate': 6.7092198581560284e-06, 'epoch': 1.0}
{'eval_loss': 0.10561616718769073, 'eval_f1': 0.6929133858267715, 'eval_runtime': 2.6015, 'eval_samples_per_second': 344.796, 'eval_steps_per_second': 28.829, 'epoch': 1.0}
{'loss': 0.0686, 'grad_norm': 3.9782965183258057, 'learning_rate': 3.375886524822695e-06, 'epoch': 2.0}
{'eval_loss': 0.09286918491125107, 'eval_f1': 0.7708333333333333, 'eval_runtime': 2.6274, 'eval_samples_per_second': 341.398, 'eval_steps_per_second': 28.545, 'epoch': 2.0}
{'loss': 0.021, 'grad_norm': 16.927980422973633, 'learning_rate': 4.2553191489361707e-08, 'epoch': 3.0}
{'eval_loss': 0.1098262295126915, 'eval_f1': 0.7878787878787878, 'eval_runtime': 3.0102, 'eval_samples_per_second': 297.984, 'eval_steps_per_second': 24.915, 'epoch': 3.0}
{'train_runtime': 222.3429, 'train_samples_per_second': 50.719, 'train_steps_per_second': 3.171, 'train_loss': 0.08156960331801827,

In [28]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch16_Lr3e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1661, 'grad_norm': 4.264297008514404, 'learning_rate': 2.0085106382978722e-05, 'epoch': 1.0}
{'eval_loss': 0.1701354831457138, 'eval_f1': 0.5681818181818182, 'eval_runtime': 2.7922, 'eval_samples_per_second': 321.258, 'eval_steps_per_second': 26.861, 'epoch': 1.0}
{'loss': 0.0888, 'grad_norm': 0.07821416109800339, 'learning_rate': 1.0127659574468085e-05, 'epoch': 2.0}
{'eval_loss': 0.16505032777786255, 'eval_f1': 0.6588235294117648, 'eval_runtime': 2.7142, 'eval_samples_per_second': 330.49, 'eval_steps_per_second': 27.633, 'epoch': 2.0}
{'loss': 0.03, 'grad_norm': 0.024444399401545525, 'learning_rate': 1.7021276595744683e-07, 'epoch': 3.0}
{'eval_loss': 0.17352549731731415, 'eval_f1': 0.6530612244897959, 'eval_runtime': 2.7023, 'eval_samples_per_second': 331.937, 'eval_steps_per_second': 27.754, 'epoch': 3.0}
{'train_runtime': 221.6066, 'train_samples_per_second': 50.887, 'train_steps_per_second': 3.181, 'train_loss': 0.0949805185304465

In [29]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00005  # 5e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch16_Lr5e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 5e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.243, 'grad_norm': 1.0816830396652222, 'learning_rate': 3.347517730496454e-05, 'epoch': 1.0}
{'eval_loss': 0.21686427295207977, 'eval_f1': 0.0, 'eval_runtime': 2.6807, 'eval_samples_per_second': 334.619, 'eval_steps_per_second': 27.978, 'epoch': 1.0}
{'loss': 0.2306, 'grad_norm': 0.7036086320877075, 'learning_rate': 1.6808510638297873e-05, 'epoch': 2.0}
{'eval_loss': 0.22009693086147308, 'eval_f1': 0.0, 'eval_runtime': 2.6566, 'eval_samples_per_second': 337.645, 'eval_steps_per_second': 28.231, 'epoch': 2.0}
{'loss': 0.2289, 'grad_norm': 0.9041065573692322, 'learning_rate': 1.4184397163120568e-07, 'epoch': 3.0}
{'eval_loss': 0.21727757155895233, 'eval_f1': 0.0, 'eval_runtime': 3.175, 'eval_samples_per_second': 282.516, 'eval_steps_per_second': 23.622, 'epoch': 3.0}
{'train_runtime': 217.7079, 'train_samples_per_second': 51.799, 'train_steps_per_second': 3.238, 'train_loss': 0.23419096926425365, 'epoch': 3.0}
----------  Res on Valid  ---

#### Batch Size -> 32

In [30]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.000003  # 3e-6

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch32_Lr3e-06
N_EPOCH:  3
BATCH_SIZE:  32
LR 3e-06


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.2001, 'grad_norm': 1.6359843015670776, 'learning_rate': 2.016949152542373e-06, 'epoch': 1.0}
{'eval_loss': 0.1057385578751564, 'eval_f1': 0.5217391304347826, 'eval_runtime': 2.693, 'eval_samples_per_second': 333.089, 'eval_steps_per_second': 27.85, 'epoch': 1.0}
{'loss': 0.0972, 'grad_norm': 13.028168678283691, 'learning_rate': 1.0254237288135593e-06, 'epoch': 2.0}
{'eval_loss': 0.08661895990371704, 'eval_f1': 0.686868686868687, 'eval_runtime': 2.8381, 'eval_samples_per_second': 316.061, 'eval_steps_per_second': 26.426, 'epoch': 2.0}
{'loss': 0.0679, 'grad_norm': 0.8082497119903564, 'learning_rate': 2.5423728813559323e-08, 'epoch': 3.0}
{'eval_loss': 0.08999272435903549, 'eval_f1': 0.6382978723404256, 'eval_runtime': 2.985, 'eval_samples_per_second': 300.5, 'eval_steps_per_second': 25.125, 'epoch': 3.0}
{'train_runtime': 151.2217, 'train_samples_per_second': 74.573, 'train_steps_per_second': 2.341, 'train_loss': 0.1217337759201136, 'epo

In [31]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch32_Lr1e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1518, 'grad_norm': 0.5459785461425781, 'learning_rate': 6.72316384180791e-06, 'epoch': 1.0}
{'eval_loss': 0.09750784933567047, 'eval_f1': 0.6373626373626374, 'eval_runtime': 3.1519, 'eval_samples_per_second': 284.587, 'eval_steps_per_second': 23.795, 'epoch': 1.0}
{'loss': 0.08, 'grad_norm': 0.08581718802452087, 'learning_rate': 3.3898305084745763e-06, 'epoch': 2.0}
{'eval_loss': 0.0890704095363617, 'eval_f1': 0.6862745098039216, 'eval_runtime': 2.6921, 'eval_samples_per_second': 333.197, 'eval_steps_per_second': 27.859, 'epoch': 2.0}
{'loss': 0.0307, 'grad_norm': 0.07916422933340073, 'learning_rate': 5.649717514124294e-08, 'epoch': 3.0}
{'eval_loss': 0.11827506124973297, 'eval_f1': 0.7, 'eval_runtime': 3.0324, 'eval_samples_per_second': 295.807, 'eval_steps_per_second': 24.733, 'epoch': 3.0}
{'train_runtime': 149.6018, 'train_samples_per_second': 75.38, 'train_steps_per_second': 2.366, 'train_loss': 0.08748723760163044, 'epoch': 3.0}
-

In [32]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch32_Lr3e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.17, 'grad_norm': 7.0553812980651855, 'learning_rate': 1.9999999999999998e-05, 'epoch': 1.0}
{'eval_loss': 0.0906379371881485, 'eval_f1': 0.6391752577319586, 'eval_runtime': 2.8414, 'eval_samples_per_second': 315.689, 'eval_steps_per_second': 26.395, 'epoch': 1.0}
{'loss': 0.0864, 'grad_norm': 0.24864795804023743, 'learning_rate': 1.0084745762711864e-05, 'epoch': 2.0}
{'eval_loss': 0.09336385875940323, 'eval_f1': 0.7142857142857142, 'eval_runtime': 2.7006, 'eval_samples_per_second': 332.149, 'eval_steps_per_second': 27.772, 'epoch': 2.0}
{'loss': 0.0302, 'grad_norm': 0.7057201862335205, 'learning_rate': 1.694915254237288e-07, 'epoch': 3.0}
{'eval_loss': 0.13734616339206696, 'eval_f1': 0.7102803738317756, 'eval_runtime': 2.6826, 'eval_samples_per_second': 334.379, 'eval_steps_per_second': 27.958, 'epoch': 3.0}
{'train_runtime': 148.1671, 'train_samples_per_second': 76.11, 'train_steps_per_second': 2.389, 'train_loss': 0.09554789699403579,

In [33]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00005  # 5e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch32_Lr5e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 5e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.2346, 'grad_norm': 0.8112380504608154, 'learning_rate': 3.361581920903955e-05, 'epoch': 1.0}
{'eval_loss': 0.21515560150146484, 'eval_f1': 0.0, 'eval_runtime': 2.666, 'eval_samples_per_second': 336.453, 'eval_steps_per_second': 28.132, 'epoch': 1.0}
{'loss': 0.2278, 'grad_norm': 0.8923736810684204, 'learning_rate': 1.694915254237288e-05, 'epoch': 2.0}
{'eval_loss': 0.22266942262649536, 'eval_f1': 0.0, 'eval_runtime': 3.0718, 'eval_samples_per_second': 292.008, 'eval_steps_per_second': 24.415, 'epoch': 2.0}
{'loss': 0.2254, 'grad_norm': 0.7918222546577454, 'learning_rate': 2.8248587570621473e-07, 'epoch': 3.0}
{'eval_loss': 0.21811603009700775, 'eval_f1': 0.0, 'eval_runtime': 2.6718, 'eval_samples_per_second': 335.731, 'eval_steps_per_second': 28.071, 'epoch': 3.0}
{'train_runtime': 149.5741, 'train_samples_per_second': 75.394, 'train_steps_per_second': 2.367, 'train_loss': 0.2292671796292235, 'epoch': 3.0}
----------  Res on Valid  ----

#### Top4の繰り返し

In [34]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch16_Lr1e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1658, 'grad_norm': inf, 'learning_rate': 6.723404255319149e-06, 'epoch': 1.0}
{'eval_loss': 0.12499923259019852, 'eval_f1': 0.6615384615384615, 'eval_runtime': 2.7822, 'eval_samples_per_second': 322.402, 'eval_steps_per_second': 26.957, 'epoch': 1.0}
{'loss': 0.0761, 'grad_norm': 0.3134479820728302, 'learning_rate': 3.4042553191489363e-06, 'epoch': 2.0}
{'eval_loss': 0.15495380759239197, 'eval_f1': 0.6942148760330579, 'eval_runtime': 2.6779, 'eval_samples_per_second': 334.969, 'eval_steps_per_second': 28.007, 'epoch': 2.0}
{'loss': 0.0323, 'grad_norm': 0.049598656594753265, 'learning_rate': 7.092198581560284e-08, 'epoch': 3.0}
{'eval_loss': 0.12020200490951538, 'eval_f1': 0.7474747474747474, 'eval_runtime': 2.8298, 'eval_samples_per_second': 316.984, 'eval_steps_per_second': 26.504, 'epoch': 3.0}
{'train_runtime': 238.3521, 'train_samples_per_second': 47.312, 'train_steps_per_second': 2.958, 'train_loss': 0.09141190576215162, 'epoch': 3

In [35]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch16_Lr3e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1721, 'grad_norm': 8.103814125061035, 'learning_rate': 2.0127659574468084e-05, 'epoch': 1.0}
{'eval_loss': 0.1390731781721115, 'eval_f1': 0.5070422535211268, 'eval_runtime': 2.6956, 'eval_samples_per_second': 332.759, 'eval_steps_per_second': 27.823, 'epoch': 1.0}
{'loss': 0.0885, 'grad_norm': 0.07130773365497589, 'learning_rate': 1.0127659574468085e-05, 'epoch': 2.0}
{'eval_loss': 0.15759478509426117, 'eval_f1': 0.6538461538461539, 'eval_runtime': 3.1994, 'eval_samples_per_second': 280.367, 'eval_steps_per_second': 23.442, 'epoch': 2.0}
{'loss': 0.03, 'grad_norm': 26.90654754638672, 'learning_rate': 1.276595744680851e-07, 'epoch': 3.0}
{'eval_loss': 0.15439938008785248, 'eval_f1': 0.7234042553191491, 'eval_runtime': 2.7002, 'eval_samples_per_second': 332.202, 'eval_steps_per_second': 27.776, 'epoch': 3.0}
{'train_runtime': 220.4409, 'train_samples_per_second': 51.157, 'train_steps_per_second': 3.198, 'train_loss': 0.09685954235969706, 

In [36]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch32_Lr1e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1433, 'grad_norm': 8.86000919342041, 'learning_rate': 6.72316384180791e-06, 'epoch': 1.0}
{'eval_loss': 0.10509336739778519, 'eval_f1': 0.6559999999999999, 'eval_runtime': 2.6984, 'eval_samples_per_second': 332.42, 'eval_steps_per_second': 27.794, 'epoch': 1.0}
{'loss': 0.0658, 'grad_norm': 0.5345315337181091, 'learning_rate': 3.3898305084745763e-06, 'epoch': 2.0}
{'eval_loss': 0.10906297713518143, 'eval_f1': 0.6896551724137931, 'eval_runtime': 2.9428, 'eval_samples_per_second': 304.817, 'eval_steps_per_second': 25.486, 'epoch': 2.0}
{'loss': 0.0267, 'grad_norm': 0.6995502710342407, 'learning_rate': 5.649717514124294e-08, 'epoch': 3.0}
{'eval_loss': 0.10842354595661163, 'eval_f1': 0.712871287128713, 'eval_runtime': 2.6779, 'eval_samples_per_second': 334.958, 'eval_steps_per_second': 28.007, 'epoch': 3.0}
{'train_runtime': 148.6691, 'train_samples_per_second': 75.853, 'train_steps_per_second': 2.381, 'train_loss': 0.07861521890607931, 'e

In [37]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  luke-japanese-large-lite_Epoch3_Batch32_Lr3e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1417, 'grad_norm': 15.400201797485352, 'learning_rate': 2.0084745762711865e-05, 'epoch': 1.0}
{'eval_loss': 0.09387703239917755, 'eval_f1': 0.6904761904761905, 'eval_runtime': 2.8902, 'eval_samples_per_second': 310.362, 'eval_steps_per_second': 25.95, 'epoch': 1.0}
{'loss': 0.0586, 'grad_norm': 2.167490005493164, 'learning_rate': 1.016949152542373e-05, 'epoch': 2.0}
{'eval_loss': 0.11248192936182022, 'eval_f1': 0.6956521739130435, 'eval_runtime': 2.6829, 'eval_samples_per_second': 334.338, 'eval_steps_per_second': 27.955, 'epoch': 2.0}
{'loss': 0.0181, 'grad_norm': 0.014353667385876179, 'learning_rate': 1.694915254237288e-07, 'epoch': 3.0}
{'eval_loss': 0.12999524176120758, 'eval_f1': 0.7115384615384615, 'eval_runtime': 3.183, 'eval_samples_per_second': 281.808, 'eval_steps_per_second': 23.563, 'epoch': 3.0}
{'train_runtime': 153.3327, 'train_samples_per_second': 73.546, 'train_steps_per_second': 2.309, 'train_loss': 0.07280329526480982